In [1]:
from src.smth2matrix.shift2zero import shift2zero
import numpy as np
from src.greedy_alg.class_pallets import Pallets
from src.class_item import Item

from copy import copy

points = np.array([[0.3, 3], [1, 0], [2.1, 0], [3, 3.7]])
shift2zero(points)

[2.7, 3.7]

In [2]:
item_1 = Item(1, points)
item_1.set_segments(1)
print(item_1.segments)


[[], [], [], [], []]
[[[0.7, 1.8]], [[0.46666666666666656, 2.043243243243243]], [[0.23333333333333328, 2.2864864864864862]], [[0.0, 2.5297297297297296]], [[2.7, 2.7]]]
[[[0.7, 1.8]], [[0.46666666666666656, 2.043243243243243]], [[0.23333333333333328, 2.2864864864864862]], [[0.0, 2.7]], [[2.7, 2.7]]]


In [ ]:
segments = [
    [[0.7, 1.8], [2,3]], [[0.46666666666666656, 2.043243243243243]], [[0.23333333333333328, 2.2864864864864862]], [[0.0, 2.5297297297297296]], [[2.7, 2.7]]
]

segments_2 = [
    [[0.7, 1.8], [2,4]], [[0.46666666666666656, 2.043243243243243]], [[0.23333333333333328, 2.2864864864864862]], [[0.0, 2.5297297297297296]], [[2.7, 2.7]]
]

item_1 = Item(1, points)

item_1.segments = segments

item_2 = Item(2, points)

item_2.segments = segments_2

items = [copy(item_1), copy(item_2)]

from src.class_packing import Packing
from src.smth2lines.polygon2segments import polygon2segments

h = 1
points = np.array([[0.3, 3], [1, 0], [2.1, 0], [3, 3.7]])

lines = polygon2segments(points, h)
print(lines)

packaging = Packing(width=200, height=30, drill_radius=0)

pallets = Pallets([20, 20])
pallets.add_pallet_lines()

print(pallets.pallet_lines)

In [2]:
for r in range(len(pallets.pallet_lines)):
    row = pallets.pallet_lines[r]
    for k in range(1, len(row)):
        for item in items:
            if not item.packed:
                t_vector = row[k-1][1] - item.segments[0][0][0]
                r_flag = 0
                ex_flag = 0
                i = 0
                l = len(item.segments[0])
                while i < len(item.segments) and r + i < len(pallets.pallet_lines) and ex_flag == 0 or ex_flag == 1:
                    j = 0
                    while j < len(item.segments[i]) and ex_flag == 0:
                        m = 0
                        while m < len(pallets.pallet_lines[r+i]) and ex_flag == 0:
                            # случаи пересечения линии и упаковки
                            if pallets.pallet_lines[r+i][m][0] <= item.segments[i][j][0] + t_vector < pallets.pallet_lines[r+i][m][1]:
                                t_vector += pallets.pallet_lines[r+i][m][1] - (item.segments[i][j][0] + t_vector)
                                i = 0
                            elif pallets.pallet_lines[r+i][m][0] <= item.segments[i][j][1] + t_vector < pallets.pallet_lines[r+i][m][1]:
                                t_vector += pallets.pallet_lines[r+i][m][1] - (item.segments[i][j][0] + t_vector)
                                i = 0
                            # объект выходит за границы палеты
                            if item.segments[i][j][0] + t_vector < 0:
                                t_vector = - item.segments[i][j][0]
                                i = 0
                            elif item.segments[i][j][1] + t_vector > pallets.shape[0]:
                                ex_flag = 1
                            m += 1
                        j += 1
                    i+=1
                # если объект влезает, добавляем его на палету
                if ex_flag == 0:
                    # print('hello', r, 'id', item.id)
                    item.t_vector = t_vector
                    item.packed = True
                    # print('вектор: ',item.t_vector)
                    for s in range(len(item.segments)):
                        # print('сегмент: ',item.segments[s])
                        for l in range(len(item.segments[s])):
                            new_line = [item.segments[s][l][0] + item.t_vector, item.segments[s][l][1] + item.t_vector]
                            pallets.pallet_lines[r+s].append(copy(new_line))
                        pallets.pallet_lines[r+s].sort(key=lambda x: int(x[0]), reverse=False)
                        print('строка', r+s , ':', pallets.pallet_lines[r+s])


строка 0 : [[0, 0], [0.7, 1.8], [2.0, 3.0], [20, 20]]
строка 1 : [[0, 0], [0.46666666666666656, 2.043243243243243], [20, 20]]
строка 2 : [[0, 0], [0.23333333333333328, 2.2864864864864862], [20, 20]]
строка 3 : [[0, 0], [0.0, 2.5297297297297296], [20, 20]]
строка 4 : [[0, 0], [2.7, 2.7], [20, 20]]
строка 0 : [[0, 0], [0.7, 1.8], [2.0, 3.0], [3.22972972972973, 4.3297297297297295], [4.52972972972973, 6.52972972972973], [20, 20]]
строка 1 : [[0, 0], [0.46666666666666656, 2.043243243243243], [2.9963963963963964, 4.5729729729729724], [20, 20]]
строка 2 : [[0, 0], [0.23333333333333328, 2.2864864864864862], [2.763063063063063, 4.816216216216215], [20, 20]]
строка 3 : [[0, 0], [0.0, 2.5297297297297296], [2.5297297297297296, 5.059459459459459], [20, 20]]
строка 4 : [[0, 0], [2.7, 2.7], [5.22972972972973, 5.22972972972973], [20, 20]]
